## MoVer Tutorial

# ![MoVer Pipeline](assets/mover_pipeline.png)

As MoVer can be used as an end-to-end animation pipeline, you can also use its various components separately. This tutorial will walk you through these components and show you how to use them. 

In case you just want to look through the code, we have included the generated results of the first four parts of this tutorial in the `examples/` directory (`translate_right*` files).

In [ ]:
from mover.dsl.mover_verifier import MoverVerifier
from mover.synthesizers.animation_synthesizer import AnimationSynthesizer
from mover.synthesizers.mover_synthesizer import MoverSynthesizer
import subprocess

## 1. Generate an animation program

In [ ]:
## Set a unique identifier for this animation for naming the output files
animation_id = "demo_translate_right"

## Define the animation we want to create using natural language
animation_prompt = "Translate the black square to the right by 100 px over 2 seconds."

## Static SVG file path to be animated
svg_file_path = "examples/svg/four_objects.svg"

In [ ]:
## Create an instance of the animation synthesizer. By default, it uses GPT-4.1 from OpenAI
## Make sure you have the API key for OpenAI in your environment variables (the name has to be `OPENAI_API_KEY`)
animation_synthesizer = AnimationSynthesizer() 

## Example: using llama-3.3-70b-versatile from Groq. Refer to the YAML config file for more details about how to pass in parameters like temperature, top_p, etc.
# animation_synthesizer = AnimationSynthesizer(model_name="llama-3.3-70b-versatile", provider="groq") 

## Initialize the chat history with system message and initial animation prompt
## The system message sets up the context for the LLM
## The initial animation prompt includes our animation description and the SVG file to animate
## You can specify compose_sys_msg(sys_msg_file_path=<some_path>) to use a custom system message
chat_history = [animation_synthesizer.compose_sys_msg(), 
                animation_synthesizer.compose_initial_user_prompt(animation_prompt, svg_file_path)]

## Generate the JavaScript animation code
response, error_msg = animation_synthesizer.generate(chat_history, 
                                                    svg_file_path, 
                                                    f"examples/{animation_id}.html")

print(error_msg if error_msg else response)

## 2. Generate a MoVer program from the animation prompt

In [ ]:
## Create an instance of the MoVer program synthesizer
mover_synthesizer = MoverSynthesizer() ## by default, it uses GPT-4.1 from OpenAI
# mover_synthesizer = MoverSynthesizer(model_name="llama-3.3-70b-versatile", provider="groq") ## change the model to llama-3.3-70b-versatile from Groq

## Prepare the system message and animation prompt for generating the MoVer program
chat_history = [mover_synthesizer.compose_sys_msg(),
                mover_synthesizer.compose_initial_user_prompt(animation_prompt, svg_file_path)]

## Generate the MoVer program specification
response, error_msg = mover_synthesizer.generate(chat_history, f"examples/{animation_id}.py")

print(error_msg if error_msg else response)

## 3. Convert the animation program to MoVer data format and render a video

In [ ]:
## Convert the HTML animation into a video file using the MoVer converter tool
## Here we use subprocess because the browser-based converter tool uses asyncio, which doesn't work well with Jupyter notebooks
## Outside of Jupyter notebooks, you can directly call the converter tool with `convert_animation()`
## Parameters: the HTML file path, the port number of the converter tool, and the flag to create a video (omit if you don't want to create a video)
subprocess.run(["python", "-m", "mover.converter.mover_converter", f"examples/{animation_id}.html", "3013", "--create-video"])

## 4. Verify the animation with the MoVer program

In [ ]:
## Define paths to the MoVer program and animation data files
program_file = f"examples/{animation_id}.py"  # The MoVer program verification file
animation_file = f"examples/{animation_id}_data.json"  # The animation data compatible with the MoVer execution engine

## Create a verifier instance and verify that the animation matches the MoVer program
## The verifier checks that all predicates in the MoVer program are satisfied by the animation
verifier = MoverVerifier()
verification_report = verifier.verify(program_file, animation_file)
print(verification_report)

## 5. Putting these togther into a pipeline

In [ ]:
## Run the pipeline on the starter example
subprocess.run(["python", "-m", "mover.pipeline", "examples/configs/config_starter.yaml"])

In [ ]:
## Run the pipeline on the teaser example
subprocess.run(["python", "-m", "mover.pipeline", "examples/configs/config_teaser.yaml"])

## 6. Compute stats on the iterations

In [ ]:
## To get stats on the iterations similar to the one in the paper, run the following command
subprocess.run(["python", "-m", "mover.stats", "examples/configs/config_starter.yaml", "examples/configs/config_teaser.yaml"])